In [4]:
from helper import *

In [5]:
categories = {'pets' : 'familie/haustiere', 'politics' : 'politik',
              'economy' : 'wirtschaft', 'travel' : 'konsum/reisen',
              'health' : 'gesundheit/medizin-krankheit', 'food' : 'ernahrung/lebensmittel',
              'parenting' : 'familie/erziehung', 'realestate' : 'wohnen/eigentum',
              'career' : 'arbeit', 'home' : 'wohnen/bauen-renovieren',
              'nature' : 'umwelt/umweltpolitik'}

In [17]:
website = 'https://www.beobachter.ch'

for category, path in categories.items():
    print(f'Working with {category} category')

    lines = []
    page_num = 1

    while True:
        url = f'{website}/{path}?page={page_num}#page'
        page = requests.get(url)    
        soup = BeautifulSoup(page.content, 'html.parser')
        
        if page_num == 1:
            try:
                limit = int(soup.find('div', class_='_2fQ8qDhH _1deHe_oF').
                            find_all('a', class_='page-loader-btn ShimA1UP')[-1].text)
                print(f'Number of pages = {limit}')
            except AttributeError:
                limit = 1
                
        
        divs = soup.find_all('div', class_='_23Qt4XY-')

        if divs == []:
            break

        for div in divs[1:]:
            sub_divs = div.find_all('a', class_='_2p02b0mw teaser-m-default')

            for sub_div in sub_divs:
                teaser_headline = sub_div.find('div', class_='_2QZpcT0M _3du5gVzi _3iAkatGy')
                teaser = sub_div.find('div', class_='_2QcjsKyb M6MeFidn')
                href = sub_div['href']
                
                # going to the page of the article to get keywords
                try:
                    article = requests.get(website + href)
                    article_soup = BeautifulSoup(article.content, 'html.parser')
                    keywords = ' '.join(article_soup.find('meta', attrs={'name':'news_keywords'})['content'].split(','))
                except TypeError:
                    keywords = ''
                    
                if None in (teaser_headline, teaser):
                    continue

                teaser_headline = teaser_headline.text.strip()
                teaser = teaser.text.strip()[:-5] #[:-5] to throw away the word "Mehr"
                
                lines.append(teaser_headline)
                lines.append(teaser)
                lines.append(keywords)

                if len(lines) % 100 == 0:
                    print(f'Scraped {len(lines)} lines')

        page_num += 1
        if page_num > limit:
            break
            
    print(f'Finished working with {category} category. Scraped {len(lines)} lines\n')
    write_to_file(lines, category, 'beobachter_ch')

Working with pets category
Number of pages = 10
Haustiere Krankheit Tierhalter Erkältung Tierhaltung Tierarzt Hausmittel


TypeError: 'NoneType' object is not subscriptable